In [1]:
import streamlit as st 
import torch
import numpy as np 
import plotly.graph_objects as go
import pandas as pd 

In [2]:
lower_bound = -2
upper_bound = 2
sample_size = 5

model_lower_bound = -3
model_upper_bound = 3


# inputs for dataset 

secret_w1 = torch.tensor(1.0)
secret_w2 = torch.tensor(1.0)

x1 = torch.linspace(lower_bound,upper_bound,sample_size)
x2 = torch.linspace(lower_bound,upper_bound,sample_size) # to manipulate the shaprpness of the loss function, play with the scale of the features.


X1_mesh ,X2_mesh = torch.meshgrid(x1,x2,indexing='ij')
X1 = X1_mesh.flatten()
X2 = X2_mesh.flatten()

y = (secret_w1*X1) + (secret_w2*X2)

df = pd.DataFrame({'X1':X1,
                   'X2':X2,
                   'y':y})

df.head()

,X1,X2,y
0,-2.0,-2.0,-4.0
1,-2.0,-1.0,-3.0
2,-2.0,0.0,-2.0
3,-2.0,1.0,-1.0
4,-2.0,2.0,0.0


In [3]:
print(X1)
print(X2)
print(y)

tensor([-2., -2., -2., -2., -2., -1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,  2.])
tensor([-2., -1.,  0.,  1.,  2., -2., -1.,  0.,  1.,  2., -2., -1.,  0.,  1.,
         2., -2., -1.,  0.,  1.,  2., -2., -1.,  0.,  1.,  2.])
tensor([-4., -3., -2., -1.,  0., -3., -2., -1.,  0.,  1., -2., -1.,  0.,  1.,
         2., -1.,  0.,  1.,  2.,  3.,  0.,  1.,  2.,  3.,  4.])


In [4]:
X1_mesh

tensor([[-2., -2., -2., -2., -2.],
        [-1., -1., -1., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.]])

In [5]:
X2_mesh

tensor([[-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.],
        [-2., -1.,  0.,  1.,  2.]])

In [6]:
X1_mesh + X2_mesh 

tensor([[-4., -3., -2., -1.,  0.],
        [-3., -2., -1.,  0.,  1.],
        [-2., -1.,  0.,  1.,  2.],
        [-1.,  0.,  1.,  2.,  3.],
        [ 0.,  1.,  2.,  3.,  4.]])

In [7]:
def generate_plot(w1,w2):

  Datapoints = go.Scatter3d(
      x = X1,
      y = X2,
      z = y,
      mode = 'markers'
  )

  plane = go.Surface(
    x = X1_mesh,
    y = X2_mesh,
    z = (w1 * X1_mesh) + (w2 * X2_mesh)    # y_hat 

  )

  layout = go.Layout(
    scene= dict(
      xaxis = dict(title='feature_1',range=[-4,4]),
      yaxis = dict(title='feature_2',range=[-4,4]),
      zaxis = dict(title='y',range=[-8,8])
    )
  )

  figure = go.Figure(data=[Datapoints,plane],layout=layout)
  
  return figure



In [20]:
generate_plot(1,1)

In [13]:
# math for loss landscape 
chain_size = 11

W1_mesh,W2_mesh = torch.meshgrid(torch.linspace(-10,10,chain_size),torch.linspace(-10,10,chain_size),indexing='ij')
print(W1_mesh.shape)
print(W2_mesh.shape)

W1 = W1_mesh.flatten()
W2 = W2_mesh.flatten()
print(W1.shape)
print(W2.shape)


torch.Size([11, 11])
torch.Size([11, 11])
torch.Size([121])
torch.Size([121])


In [14]:
losses = []

for w1,w2 in zip(W1,W2):
  y_hat = X1*w1 + X2*w2
  loss = torch.mean((y - y_hat)**2)
  losses.append(loss)

In [15]:
print(len(losses))
losses_T = torch.tensor(losses)


121


In [16]:
y_hat = X1*1 + X2*1
loss = torch.mean((y - y_hat)**2)
loss

tensor(0.)

In [17]:
torch.mean( (y - (1*X1 + 1*X2))**2)

tensor(0.)

In [18]:
def loss_landscape(w1,w2):

  grid = go.Surface(
    x = W1_mesh,
    y = W2_mesh,
    z = losses_T.reshape(W1_mesh.shape),
    name = "loss functions landscape"
  )

  Global_minima = go.Scatter3d(
    x = (secret_w1,),
    y = (secret_w2,),
    z = (torch.min(losses_T),),
    mode = 'markers',
    marker = dict(color='yellow',size=10,symbol='diamond'),
    name = 'Global minima'
  )

  ball = go.Scatter3d(
    x = (w1,),
    y = (w2,),
    z = (torch.mean( (y - (w1*X1 + w2*X2))**2),),
    mode = 'markers',
    marker = dict(color='red',size = 7),
    name = 'loss'

  )

  figure = go.Figure(data = [grid,Global_minima,ball])

  return figure 


In [19]:
loss_landscape(4,4)